<a href="https://colab.research.google.com/github/sunilneupane27/week9/blob/main/with_pooling_week_5pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10

# Load and preprocess CIFAR-10 data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

# ---------------------------------------------------------------
# CNN Model with Pooling
# ---------------------------------------------------------------
def build_cnn_with_pooling():
    inputs = layers.Input(shape=(32, 32, 3))
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='last_conv')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(10, activation='softmax')(x)
    model = Model(inputs, outputs)
    return model


In [ ]:
def compute_gradcam(model, image, label_index, layer_name='last_conv'):
    grad_model = Model(inputs=model.inputs,
                       outputs=[model.get_layer(layer_name).output, model.output])

    with tf.GradientTape() as tape:
        inputs = tf.cast(tf.expand_dims(image, axis=0), tf.float32)
        conv_outputs, predictions = grad_model(inputs)
        loss = predictions[:, label_index]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
model_pool = build_cnn_with_pooling()
model_pool.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])
print("\nTraining CNN with Pooling...\n")
model_pool.fit(X_train, y_train, epochs=5, batch_size=128, validation_split=0.1)

model_nopool = build_cnn_without_pooling()
model_nopool.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])
print("\nTraining CNN without Pooling...\n")
model_nopool.fit(X_train, y_train, epochs=5, batch_size=128, validation_split=0.1)

# ---------------------------------------------------------------
# Evaluate on Test Set
# ---------------------------------------------------------------
test_loss_pool, test_acc_pool = model_pool.evaluate(X_test, y_test, verbose=0)
test_loss_nopool, test_acc_nopool = model_nopool.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Accuracy (With Pooling): {test_acc_pool*100:.2f}%")
print(f"Test Accuracy (Without Pooling): {test_acc_nopool*100:.2f}%")


Training CNN with Pooling...

Epoch 1/5
352/352 ━━━━━━━━━━━━━━━━━━━━ 97s 271ms/step - accuracy: 0.3477 - loss: 1.7738 - val_accuracy: 0.5712 - val_loss: 1.2041
Epoch 2/5
147/352 ━━━━━━━━━━━━━━━━━━━━ 54s 268ms/step - accuracy: 0.5760 - loss: 1.1879